<a href="https://colab.research.google.com/github/18ALI07/Parallel-Genetic-Algorithm/blob/main/201080078_PC_Exp_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update && apt-get install -y openmpi-bin libopenmpi-dev

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,581 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packages [1,218 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108

Parallel Genetic Algorithm

In [ ]:
%%writefile exp10.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <pthread.h>
#include <omp.h>
#include <limits.h>

//#define NUM_THREADS 16
int NUM_THREADS = 1;

const char *ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ";
const char *TARGET = "Hello, World!"; //Set the target
const int GENE_POOL_SIZE = 10; //Set the size of the gene pool
const int FIT_PARENT_PROB = 90;
const int MUTATION_PROB = 100;
int TARGET_LEN;

/* We need to keep keys and values */
typedef struct{
  char* dna;
  int fitness;
} candidate;

//---------PROTOTYPES---------
void test();
char *gen_string(int length);
int get_fitness(char *source);
int candidate_comparator(const void *arg1, const void *arg2);
candidate get_rand_parent(candidate genepool[]);
void print_candidate(candidate cand);
char *crossover(candidate parent1, candidate parent2);
void mutate(char *source);
int run();

//---------MAIN---------
int main() {
  srand(time(NULL));                            //Seed the random time
  struct timespec tstart={0,0}, tend={0,0};     //Init struct for recording time

  //Manually Setting the String
  //TARGET_LEN = strlen(TARGET);                  //Get target length

  //Random String
  TARGET_LEN = 5;
  TARGET = gen_string(TARGET_LEN);
  
  int MAX_THREADS = 64;                        //Will iteratie from 1 to MAX_THREADS
  int runs_per_thread_iter = 10;               //How many runs will be done per thread iter

  printf("THREADS, # GENERATIONS (%d avg)\n", runs_per_thread_iter);
  //Iterate from 1 - MAX_THREADS and run the alg runs_per_thread_iter amount of times each iter
  for(NUM_THREADS = 1; NUM_THREADS <= MAX_THREADS; NUM_THREADS++){
    omp_set_num_threads(NUM_THREADS);           //Set the number of threads    
    int num_generations;                        //holds the result from run()
    double elapsed_time = 0;                    //Used for statistics of each run
    int sum = 0;                                //Used to calculate average

    //Run it runs_per_thread_iter amount of times to get more accurate result
    for(int i = 0; i < runs_per_thread_iter; i++) {
      clock_gettime(CLOCK_MONOTONIC, &tstart);  //START TIMER

      num_generations = run();                  //RUN THE ALGORITHM
      sum += num_generations;                   //ADD result to the sum
      clock_gettime(CLOCK_MONOTONIC, &tend);    //END TIMER
        
      // compute and print the elapsed time in millisec
      elapsed_time += (((double)tend.tv_sec + 1.0e-9*tend.tv_nsec) - ((double)tstart.tv_sec + 1.0e-9*tstart.tv_nsec));      
    }

    //STATISTICS
    double avg_generations = (double)sum/runs_per_thread_iter;
    double avg_elapsed_time = (elapsed_time/(double)runs_per_thread_iter)*1000000.0;
    
    printf("%d,%.2f\n", NUM_THREADS, avg_generations); //For CSV output
    //printf("[THREADS: %d]: [AVG GENS: %.2f] [AVG TIME: %.2f]\n",NUM_THREADS, avg_generations, avg_elapsed_time);    
  }
}

int run() {
    
  //Generate the gene pool
  candidate genepool[GENE_POOL_SIZE];
  for(int i = 0; i < GENE_POOL_SIZE; i++){
    //genepool[i] = gen_string(target_len); //generate random string
    char *rand_dna = gen_string(TARGET_LEN); //create a random dna
    int fitval = get_fitness(rand_dna);
    candidate new_cand = {rand_dna, fitval}; //create a new candidate
    genepool[i] = new_cand; //add candidate to gene pool
  }
    
  int generation = 0;
  while(1){
    generation++;
    //Sort the genepool by fitness
    qsort(genepool, GENE_POOL_SIZE, sizeof(candidate), candidate_comparator);
    //printf("[GEN %d] ", generation); print_candidate(genepool[0]); //DEBUGING
        
    if(genepool[0].fitness == 0){
      //Target Reached
      //printf("TARGET REACHED: \"%s\" in %d generations\n", TARGET, generation); //UNCOMMENT WHEN BENCHMARKING
      return generation;
    } else {

      candidate best_child = {NULL, INT_MAX};
      #pragma omp parallel shared(best_child)
      {
	//Select two random parents
	candidate parent1 = get_rand_parent(genepool);
	candidate parent2 = get_rand_parent(genepool);
            
	//Create child by crossing over the two parents
	char *child_dna = crossover(parent1, parent1);
	mutate(child_dna);
	int child_fitval = get_fitness(child_dna);
	candidate child = {child_dna, child_fitval};
        #pragma omp critical (best_child)
	{
	  if(child.fitness < best_child.fitness){
	    best_child = child;
	  }
	}
      }//Implicit Barrier
            
      //Check if the child is better than the worst person in the genepool
      candidate lowest_cand = genepool[GENE_POOL_SIZE -1 ];
      if(best_child.fitness < lowest_cand.fitness){
	genepool[GENE_POOL_SIZE -1 ] = best_child;
      }
    }
  }
    
  //test();
  return 0;
}

//---------UTILITY FUNCTIONS---------

char *gen_string(int length){
  //Generates a random string of the given length
  char *rand_string = (char *) malloc(length); //Allocate memory for the string to be returned
  int alphabet_len = strlen(ALPHABET); //Get the length of the alphabet
    
  //Iterate length amount of times
  for(int i = 0; i < length; i++){
    int rand_index = rand() % alphabet_len; //generate random index
    rand_string[i] = ALPHABET[rand_index];  //get random character from the alphabet
  }
  //printf("%s\n", rand_string); //DEBUG
  return rand_string;
}

int get_fitness(char *source){
  //Calculates the difference between the source and target strings
  int fitval = 0;
  for(int i = 0; i < strlen(TARGET); i++){
    int curr_fitval = abs(source[i] - TARGET[i]);
    fitval += curr_fitval;
    //printf("source = %c | target = %c | diff = %d\n", source[i], target[i], curr_fitval);
  }
  //printf("TOTAL fitval = %d\n", fitval);
  return fitval;
}

// Normalized + probability-sensitive selection from the two halves of the genepool
candidate get_rand_parent(candidate genepool[]){
  int rand_index = 0;
  if(rand()% 100 < FIT_PARENT_PROB) {
    double rand_double1 = (double)rand() / (double)RAND_MAX;
    double rand_double2 = (double)rand() / (double)RAND_MAX;
    rand_index = (int) (rand_double1 * rand_double2 * ((GENE_POOL_SIZE/2) -1));
  }else{
    rand_index = (rand()%(GENE_POOL_SIZE/2)) + ((GENE_POOL_SIZE/2) -1);
  }
  return genepool[rand_index];
}

void print_candidate(candidate cand){
  printf("%s : [%d] \n", cand.dna, cand.fitness);
}

char *crossover(candidate parent1, candidate parent2){
    
  // make a copy of parent 1, as child_dna
  int length = strlen(parent1.dna);
  char *child_dna	= (char *) malloc(length);
  strcpy(child_dna, parent1.dna);
    
  // select start index for parent 2 to be inserted (1 to parent2 length -1)
  int start = (rand()%(length-1)) + 1;
    
  // select stop index for parent 2 to end insertion (2 to parent2 length -2)
  int end = (rand()%(length-2)) + 2;
    
  // if start > end, then invert them
  if (start > end) {
    int tmp = start;
    start = end;
    end = tmp;
  }
    
  // set the defined portion of the child_dna as the parent2 dna
  for(int i = start; i <= end; i++) {
    child_dna[i] = parent2.dna[i];
  }
    
  return child_dna;
}

void mutate(char *source){
    
  if(rand()% 100 < MUTATION_PROB) {
    int rand_i = rand()%strlen(source);
    source[rand_i] = source[rand_i] + (rand()%20 - 10);
  }
    
}

//---------CANDIDATE COMPARATOR---------
int candidate_comparator(const void *arg1, const void *arg2){
  const candidate *c1 = (candidate *)arg1;
  const candidate *c2 = (candidate *)arg2;
  if(c1->fitness < c2->fitness){
    return -1;
  }else if(c1->fitness > c2->fitness){
    return 1;
  }else{
    return 0;
  }
}

void test() {
  printf("test() called\n");
}

Writing exp10.c


In [ ]:
!mpicc -fopenmp -o exp10 exp10.c -lm

In [ ]:
!mpirun -np 3 --allow-run-as-root --oversubscribe ./exp10 


THREADS, # GENERATIONS (10 avg)
1,332.90
THREADS, # GENERATIONS (10 avg)
1,332.90
THREADS, # GENERATIONS (10 avg)
1,332.90
2,197.60
2,175.30
3,137.50
3,139.90
2,192.80
4,84.60
4,90.30
5,83.80
5,82.10
3,126.40
6,70.30
6,65.50
7,67.80
4,91.00
8,63.30
7,59.80
9,45.50
5,80.40
8,54.90
10,48.70
11,43.40
6,68.90
9,44.20
12,50.40
10,53.90
7,64.60
13,41.70
14,37.10
8,62.50
11,53.20
9,53.20
15,43.50
10,45.20
12,52.50
16,37.90
13,33.20
11,46.30
17,32.70
14,46.80
18,26.20
12,46.30
15,39.80
19,30.60
13,39.60
16,35.30
20,31.30
17,41.10
14,42.20
18,31.10
21,32.00
15,33.80
19,30.50
22,32.20
20,29.60
16,35.00
23,31.30
21,29.50
17,34.00
24,30.80
22,36.20
25,27.50
18,32.70
23,29.80
19,30.40
26,30.60
20,33.90
24,29.00
27,29.00
21,33.00
25,28.80
28,31.00
22,33.10
26,26.60
29,29.10
27,28.40
23,32.20
30,28.40
28,27.00
24,30.10
31,24.50
32,25.20
25,30.70
29,29.00
33,26.60
26,28.50
30,27.70
34,28.30
27,26.90
31,25.20
35,24.90
28,25.30
36,26.30
32,27.10
29,27.40
33,27.10
37,25.20
30,30.40
38,24.70
34,25.10
39,2

In [ ]:
%%writefile exp10a.c++
#include <iostream>
#include <cstdlib>
#include <cstring>
#include <ctime>
#include <cmath>
#include <limits.h>
#include <omp.h>
#include <algorithm> // Include the <algorithm> header for sort

const char *ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~ ";
const char *TARGET = "Hello, World!"; // Set the target
const int GENE_POOL_SIZE = 10;        // Set the size of the gene pool
const int FIT_PARENT_PROB = 90;
const int MUTATION_PROB = 100;
int TARGET_LEN;

// We need to keep keys and values
struct Candidate {
    char* dna;
    int fitness;
};

// Utility function to generate a random string of given length
char* genString(int length) {
    char* randString = new char[length + 1];
    int alphabetLen = strlen(ALPHABET);

    for (int i = 0; i < length; i++) {
        int randIndex = rand() % alphabetLen;
        randString[i] = ALPHABET[randIndex];
    }

    randString[length] = '\0';
    return randString;
}

// Utility function to calculate the fitness value of a candidate
int getFitness(const char* source) {
    int fitness = 0;
    int targetLen = strlen(TARGET);

    for (int i = 0; i < targetLen; i++) {
        int currFitVal = abs(source[i] - TARGET[i]);
        fitness += currFitVal;
    }

    return fitness;
}

// Normalized + probability-sensitive selection from the two halves of the gene pool
Candidate getRandParent(Candidate* genePool, int genePoolSize) {
    int randIndex = 0;

    if (rand() % 100 < FIT_PARENT_PROB) {
        double randDouble1 = (double)rand() / (double)RAND_MAX;
        double randDouble2 = (double)rand() / (double)RAND_MAX;
        randIndex = (int)(randDouble1 * randDouble2 * ((genePoolSize / 2) - 1));
    } else {
        randIndex = (rand() % (genePoolSize / 2)) + ((genePoolSize / 2) - 1);
    }

    return genePool[randIndex];
}

// Utility function to perform crossover between two parents
char* crossover(const Candidate& parent1, const Candidate& parent2) {
    int length = strlen(parent1.dna);
    char* childDna = new char[length + 1];
    strcpy(childDna, parent1.dna);

    int start = (rand() % (length - 1)) + 1;
    int end = (rand() % (length - 2)) + 2;

    if (start > end) {
        int tmp = start;
        start = end;
        end = tmp;
    }

    for (int i = start; i <= end; i++) {
        childDna[i] = parent2.dna[i];
    }

    return childDna;
}

// Utility function to perform mutation on a candidate
void mutate(char* source) {
    if (rand() % 100 < MUTATION_PROB) {
        int randIndex = rand() % strlen(source);
        source[randIndex] = source[randIndex] + (rand() % 20 - 10);
    }
}

int runGeneticAlgorithm(int numThreads) {
    srand(time(NULL));
    omp_set_num_threads(numThreads);

    TARGET_LEN = strlen(TARGET);
    int maxThreads = omp_get_max_threads();
    int runsPerThreadIter = 10;
    
    printf("THREADS, # GENERATIONS (%d avg)\n", runsPerThreadIter);

    for (int numThreads = 1; numThreads <= maxThreads; numThreads++) {
        int numGenerations;
        double elapsedAvgTime = 0;
        int sum = 0;

        for (int i = 0; i < runsPerThreadIter; i++) {
            double start = omp_get_wtime();

            Candidate genePool[GENE_POOL_SIZE];

            #pragma omp parallel for
            for (int i = 0; i < GENE_POOL_SIZE; i++) {
                char* randDna = genString(TARGET_LEN);
                int fitVal = getFitness(randDna);
                Candidate newCandidate = { randDna, fitVal };
                genePool[i] = newCandidate;
            }

            int generation = 0;

            while (true) {
                generation++;
                std::sort(genePool, genePool + GENE_POOL_SIZE, [](const Candidate& c1, const Candidate& c2) {
                    return c1.fitness < c2.fitness;
                });

                if (genePool[0].fitness == 0) {
                    // Target reached
                    // printf("TARGET REACHED: \"%s\" in %d generations\n", TARGET, generation);
                    numGenerations = generation;
                    break;
                } else {
                    Candidate bestChild = { nullptr, INT_MAX };

                    #pragma omp parallel shared(bestChild)
                    {
                        Candidate parent1 = getRandParent(genePool, GENE_POOL_SIZE);
                        Candidate parent2 = getRandParent(genePool, GENE_POOL_SIZE);

                        char* childDna = crossover(parent1, parent2);
                        mutate(childDna);
                        int childFitVal = getFitness(childDna);
                        Candidate child = { childDna, childFitVal };

                        #pragma omp critical (bestChild)
                        {
                            if (child.fitness < bestChild.fitness) {
                                bestChild = child;
                            }
                        }
                    }

                    Candidate lowestCandidate = genePool[GENE_POOL_SIZE - 1];

                    if (bestChild.fitness < lowestCandidate.fitness) {
                        genePool[GENE_POOL_SIZE - 1] = bestChild;
                    }
                }
            }

            double end = omp_get_wtime();
            double elapsedTime = (end - start) * 1000000.0;
            elapsedAvgTime += elapsedTime;
            sum += numGenerations;
        }

        double avgGenerations = (double)sum / runsPerThreadIter;
        double avgElapsedTime = elapsedAvgTime / runsPerThreadIter;

        printf("%d,%.2f\n", numThreads, avgGenerations);
        // printf("[THREADS: %d]: [AVG GENS: %.2f] [AVG TIME: %.2f]\n", numThreads, avgGenerations, avgElapsedTime);
    }

    return 0;
}

int main() {
    runGeneticAlgorithm(1); // Run the genetic algorithm with a single thread
    return 0;
}


Writing exp10a.c++


In [ ]:
!mpicc -fopenmp -o exp10a exp10a.c++ -lm

/usr/bin/ld: /tmp/ccfq2urc.o: in function `genString(int)':
exp10a.c++:(.text+0x1b): undefined reference to `operator new[](unsigned long)'
/usr/bin/ld: /tmp/ccfq2urc.o: in function `crossover(Candidate const&, Candidate const&)':
exp10a.c++:(.text+0x235): undefined reference to `operator new[](unsigned long)'
/usr/bin/ld: /tmp/ccfq2urc.o: in function `__static_initialization_and_destruction_0(int, int)':
exp10a.c++:(.text+0x13b4): undefined reference to `std::ios_base::Init::Init()'
/usr/bin/ld: exp10a.c++:(.text+0x13c9): undefined reference to `std::ios_base::Init::~Init()'
/usr/bin/ld: /tmp/ccfq2urc.o:(.data.rel.local.DW.ref.__gxx_personality_v0[DW.ref.__gxx_personality_v0]+0x0): undefined reference to `__gxx_personality_v0'
collect2: error: ld returned 1 exit status


In [ ]:
!mpirun -np 3 --allow-run-as-root --oversubscribe ./exp10a 

--------------------------------------------------------------------------
mpirun was unable to launch the specified application as it could not access
or execute an executable:

Executable: ./exp10a
Node: 8b793ee91ff9

while attempting to start process rank 0.
--------------------------------------------------------------------------
3 total processes failed to start


Serial Genetic algorithm implementation

In [ ]:
%%writefile exp10b.c++
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <cmath>

const int POPULATION_SIZE = 100;
const int CHROMOSOME_LENGTH = 10;
const double TARGET_FITNESS = 0.0;
const int MAX_GENERATIONS = 1000;
const double MUTATION_RATE = 0.01;

// Structure to represent an individual in the population
struct Individual {
    std::vector<int> chromosome;
    double fitness;

    Individual() {
        chromosome.resize(CHROMOSOME_LENGTH);
        fitness = 0.0;
    }
};

// Function to generate a random individual
Individual generateIndividual() {
    Individual individual;
    for (int i = 0; i < CHROMOSOME_LENGTH; i++) {
        individual.chromosome[i] = rand() % 2; // Random binary value
    }
    return individual;
}

// Function to calculate the fitness of an individual
double calculateFitness(const Individual& individual) {
    double fitness = 0.0;
    for (int i = 0; i < CHROMOSOME_LENGTH; i++) {
        if (individual.chromosome[i] == 1) {
            fitness += 1.0; // Increment fitness if gene is 1
        }
    }
    return std::abs(fitness - TARGET_FITNESS);
}

// Function to perform crossover between two parents
Individual crossover(const Individual& parent1, const Individual& parent2) {
    Individual child;
    int crossoverPoint = rand() % CHROMOSOME_LENGTH;
    for (int i = 0; i < CHROMOSOME_LENGTH; i++) {
        if (i < crossoverPoint) {
            child.chromosome[i] = parent1.chromosome[i];
        } else {
            child.chromosome[i] = parent2.chromosome[i];
        }
    }
    return child;
}

// Function to perform mutation on an individual
void mutate(Individual& individual) {
    for (int i = 0; i < CHROMOSOME_LENGTH; i++) {
        if (rand() / double(RAND_MAX) < MUTATION_RATE) {
            individual.chromosome[i] = 1 - individual.chromosome[i]; // Flip the bit
        }
    }
}

// Function to select two parents from the population using tournament selection
std::pair<Individual, Individual> selectParents(const std::vector<Individual>& population) {
    int index1 = rand() % POPULATION_SIZE;
    int index2 = rand() % POPULATION_SIZE;
    return std::make_pair(population[index1], population[index2]);
}

// Function to evolve the population for one generation
void evolve(std::vector<Individual>& population) {
    std::vector<Individual> newPopulation;
    for (int i = 0; i < POPULATION_SIZE; i++) {
        std::pair<Individual, Individual> parents = selectParents(population);
        Individual child = crossover(parents.first, parents.second);
        mutate(child);
        child.fitness = calculateFitness(child);
        newPopulation.push_back(child);
    }
    population = newPopulation;
}

// Function to find the best individual in the population
Individual findBestIndividual(const std::vector<Individual>& population) {
    Individual bestIndividual = population[0];
    for (int i = 1; i < POPULATION_SIZE; i++) {
        if (population[i].fitness < bestIndividual.fitness) {
            bestIndividual = population[i];
        }
    }
    return bestIndividual;
}

int main() {
    srand(static_cast<unsigned>(time(0)));

    // Initialize population
    std::vector<Individual> population(POPULATION_SIZE);
    for (int i = 0; i < POPULATION_SIZE; i++) {
        population[i] = generateIndividual();
        population[i].fitness = calculateFitness(population[i]);
    }

    int generation = 0;
    while (generation < MAX_GENERATIONS) {
        evolve(population);
        Individual bestIndividual = findBestIndividual(population);

        std::cout << "Generation: " << generation << " | Best Fitness: " << bestIndividual.fitness << std::endl;

        if (bestIndividual.fitness <= 0.0) {
            std::cout << "Target fitness reached!" << std::endl;
            break;
        }

        generation++;
    }

    return 0;
}


Writing exp10b.c++


In [ ]:

!g++ -o exp10b exp10b.c++ 

In [ ]:
!./exp10b

Generation: 0 | Best Fitness: 1
Generation: 1 | Best Fitness: 0
Target fitness reached!
